In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../data/Housing.csv")
df.sample(3)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
374,3640000,3520,2,2,1,yes,no,yes,no,no,0,no,semi-furnished
56,7343000,11440,4,1,2,yes,no,yes,no,no,1,yes,semi-furnished
312,4098500,3600,3,1,1,yes,no,yes,no,yes,0,yes,furnished


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [4]:
df['hotwaterheating'].value_counts()

hotwaterheating
no     520
yes     25
Name: count, dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

for col in df.columns:
    if df[col].dtype == 'O':
            df[col] = encoder.fit_transform(df[col])

df.sample(10)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
340,3850000,5300,5,2,2,1,0,0,0,0,0,0,1
409,3430000,3185,2,1,1,1,0,0,0,0,2,0,2
40,7875000,6550,3,1,2,1,0,1,0,1,0,1,0
338,3885000,3780,2,1,2,1,1,1,0,0,0,0,1
23,8645000,4560,3,2,2,1,1,1,0,1,1,0,0
97,6300000,6400,3,1,1,1,1,1,0,1,1,1,1
19,8855000,6420,3,2,2,1,0,0,0,1,1,1,1
298,4200000,5800,3,1,1,1,0,0,1,0,2,0,1
535,2100000,3360,2,1,1,1,0,0,0,0,1,0,2
159,5460000,3150,3,2,1,1,1,1,0,1,0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   price             545 non-null    int64
 1   area              545 non-null    int64
 2   bedrooms          545 non-null    int64
 3   bathrooms         545 non-null    int64
 4   stories           545 non-null    int64
 5   mainroad          545 non-null    int32
 6   guestroom         545 non-null    int32
 7   basement          545 non-null    int32
 8   hotwaterheating   545 non-null    int32
 9   airconditioning   545 non-null    int32
 10  parking           545 non-null    int64
 11  prefarea          545 non-null    int32
 12  furnishingstatus  545 non-null    int32
dtypes: int32(7), int64(6)
memory usage: 40.6 KB


In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

x, y = df.iloc[:, :-1].values, df.iloc[:, -1].values

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)
dtr = DecisionTreeRegressor()
knnr = KNeighborsRegressor()
lr = LinearRegression()

print(np.mean(cross_val_score(dtr, x, y, cv=10, scoring='r2')))
print(np.mean(cross_val_score(knnr, x, y, cv=10, scoring='r2')))
print(np.mean(cross_val_score(lr, x, y, cv=10, scoring='r2')))

-1.4889602795162211
-0.2511363868931095
-0.1893810000980971


### Bagging

In [8]:
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVC

bag = BaggingRegressor(KNeighborsRegressor(), random_state=42)
bag.fit(xtrain, ytrain)

ypred = bag.predict(xtest)
bag.score(xtest, ytest)

-0.08108577955454033

In [9]:
params = {
            # 'base_estimator':[None, KNeighborsRegressor(), LinearRegression()],
            'n_estimators':[1000, 2000, 1500, 1000],
            'max_samples':[0.5,0.75,1.0],
            'max_features':[0.5, 1.0],
            'bootstrap':[True, False],
            'bootstrap_features':[True, False]
}

In [10]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(BaggingRegressor(random_state=1, n_jobs=-1), param_grid=params, cv=3, n_jobs = -1, verbose=1)
gs.fit(xtrain, ytrain)
gs.best_score_

Fitting 3 folds for each of 96 candidates, totalling 288 fits


0.09920510967981873

In [11]:
gs = GridSearchCV(BaggingRegressor(KNeighborsRegressor(), random_state=1, n_jobs=-1), param_grid=params, cv=3, n_jobs = -1, verbose=1)
gs.fit(xtrain, ytrain)
gs.best_score_

Fitting 3 folds for each of 96 candidates, totalling 288 fits


0.10768037178336698

In [12]:
gs = GridSearchCV(BaggingRegressor(LinearRegression(), random_state=1, n_jobs=-1), param_grid=params, cv=3, n_jobs = -1, verbose=1)
gs.fit(xtrain, ytrain)
gs.best_score_

Fitting 3 folds for each of 96 candidates, totalling 288 fits


0.08253236646885875